One dataset fits well but lacka age information (GSE152418)
        Werde  nicht alles mappen können, habe zwar eine xlsx file von flo bekommen, dabei sind aber nicht alle samples enthalten (nCov034) z.b. nicht... denke ich kann ein mapping hinbekommen, aber mit abstrichen
The other one is single-cell but does not lack ages (fokus for now is bulk)

In [ ]:
import pandas as pd
import os
import sys
from scipy.io import mmread
sys.path.append(os.path.abspath(os.path.join('..', 'utils')))
from geo_utils import read_prefixes_from_metadata_txt
from filepath import *


In [ ]:
counts_gse152418 = pd.read_csv(PATH_ARUNACHALAM2020_RAWDATA, sep='\t')
counts_gse152418_T = counts_gse152418.T
counts_gse152418_T.columns = counts_gse152418_T.iloc[0,:]
counts_gse152418_T = counts_gse152418_T.drop(counts_gse152418_T.index[0])
print(counts_gse152418.shape)

counts_gse152418_T.head()

In [ ]:

metadata_txt_GSE152418 = read_prefixes_from_metadata_txt(PATH_ARUNACHALAM2020_GEOMETADATA)
print(metadata_txt_GSE152418.shape)
metadata_txt_GSE152418.head()

In [ ]:
merged_df_GSE152418 = pd.merge(metadata_txt_GSE152418, counts_gse152418_T, left_on='title', right_index=True)

In [ ]:
print(merged_df_GSE152418.shape)
merged_df_GSE152418.head()


In [ ]:
merged_df_GSE152418.to_csv(PATH_ARUNACHALAM2020_FINALDESTINATION)

In [ ]:
merged_df_GSE152418[['ID_1', 'ID_2']] = merged_df_GSE152418['title'].str.split('_', expand=True, n=1)
merged_df_GSE152418.head()

In [ ]:

#### Then i need mapping for the ESEMBLEID´s to gene descriptors. I used this: https://www.biotools.fr/mouse/ensembl_symbol_converter 
# TODO @Elias Schreiner: please rename after you removed your home dir from the path!
ensambleid_mapping = pd.read_csv('../utils/ENSEMBLID_mapping.txt', sep='\t')
mapping_dict_ensemblid = ensambleid_mapping.set_index('ENSEMBLID')['Gene'].to_dict()
counts_gse152418_T.columns = counts_gse152418_T.columns.map(mapping_dict_ensemblid)


In [ ]:
print(counts_gse152418_T.shape)
counts_gse152418_T.head()

In [ ]:
### I got data per mail from Flo Wimmers:

# TODO @Elias Schreiner: please rename after you removed your home dir from the path!
metadata_flo_gse152418_path = "/home/eliasschreiner/PROJEKT/2020_Arunachalam_Wimmers/arunachalam_subjects.xlsx"
metadata_gse152418 = pd.read_excel(metadata_flo_gse152418_path)
print(metadata_gse152418.shape)
print(f"Unique patient ids in metadata: {len(list(metadata_gse152418['Patient_ID'].unique()))}")
metadata_gse152418.iloc[74:111,:]

metadata_gse152418.head()
metadata_gse152418.iloc[73:112,:].head()
# Conditionally transform the 'original_id' column to match the 'ID_2' format only if it contains "(Draw X)"
metadata_gse152418['ID_2'] = metadata_gse152418['Patient_ID'].astype(str).apply(
  lambda x: x if pd.isnull(x) or '(Draw ' not in x else x.replace(' (Draw ', '-Draw-').replace(')', '')
)


In [ ]:
merged_df_GSE152418_w_age_df_left = pd.merge(merged_df_GSE152418, metadata_gse152418, on='ID_2', how='left', indicator=True)
print(merged_df_GSE152418_w_age_df.shape)
merged_df_GSE152418_w_age_df[merged_df_GSE152418_w_age_df['_merge'] == 'left_only']['title']
# unmergeable: 
''' 
0               S145_nCOV001_C
7     S154_nCoV0010EUHM-Draw-1
8             S155_nCOV021EUHM
9      S156_nCOV024EUHM-Draw-1
10           S157_nCOV0029EUHM
11     S175_nCoV024EUHM-Draw-2
15     S179_nCoV033EUHM-Draw-1
16     S180_nCoV034EUHM-Draw-1
30                 S182_SHXA10
32                 S184_SHXA18
34                 S186_SHXA14
'''
### I guess i could manually add: S145_nCOV001_C, S154_nCoV0010EUHM-Draw-1 and S157_nCOV0029EUHM(probably a type and a 0 was added), S155_nCOV021EUHM (as there is only 1 draw), 
merged_df_GSE152418_w_age_df_matches = merged_df_GSE152418_w_age_df[merged_df_GSE152418_w_age_df['_merge'] != 'left_only']
print(merged_df_GSE152418_w_age_df_matches.shape)
merged_df_GSE152418_w_age_df_matches.head()

In [ ]:
merged_df_GSE152418_w_age_df_matches.to_csv("2020_aranachalam_GSE152418_combi.csv")

In [ ]:
metadata_gse152418




# ### Metadata from the SRA Run tool -> i feel like this metadata is wrong... . Currently the SRA selector tool lacka age... 
# -> Flo fragen ob er hier noch infos zu der subjects ages hat 


## was for the other GSE sample which is single cell. .
# metadata_file = gse152418_folder + '/metadata.txt'
# metadata_gse152418 = pd.read_csv(metadata_file)
# print(metadata_gse152418.shape)
# metadata_gse152418.head()


In [ ]:
### Oooookay my mapping did not work out, the GSM from meta differs from the downloaded table
metadata_gse152418[metadata_gse152418['Sample Name']=='GSM4614987']

In [ ]:
#### IMPORTANT; We merge per sample, NOT per subject here! 
combi_df_all_meta = pd.merge(counts_gse152418_T, metadata_gse152418, left_on='ID', right_on='Sample Name', how='left')
print(combi_df_all_meta.shape)
combi_df_all_meta.head()
#### großen all meta file saven
#### reduzieren um mit 10kimmu zu matchen


#### dann evtl "optimieren"


In [ ]:
totalseq_df = pd.read_csv(gse152418_folder + 'geo/.csv')


In [ ]:
## Biomarkers from 
print(f'Biomarkers: {len(totalseq_df['Specificity'].unique())}')
spec_names = totalseq_df['Specificity'].unique()
totalseq_df['Specificity'].unique()
# what is: 
## Catalog (table headline), and Clone ?
## κ Isotype Control ?
# Not important but: Can you even count an antibody as a biomarker ? 
# Count try: {entry: totalseq_df['Specificity'].value_counts().get(entry, 0) for entry in spec_names}

In [ ]:
# TODO @Elias Schreiner: please rename after you removed your home dir from the path!
features_df = pd.read_csv(gse152418_folder + 'geo/GSE155673_features.tsv', sep='\t')
print(features_df.shape)
features_df.head()
#### HAS NO COLUMNSNAMES -> 1. Gene ID Ensample id, 2. Gene symbol , 3. Descriptor ?
### scTranscriptsome pro proble gibts pro zelle einen wert.
###### spalte: sc 

In [ ]:

# TODO @Elias Schreiner: please rename after you removed your home dir from the path!
barcodes_df = pd.read_csv(gse152418_folder + 'geo/GSE155673_barcodes.tsv', sep='\t')
barcodes_df.head()

In [ ]:
### COV 1 -> this is a subject

# TODO @Elias Schreiner: please rename after you removed your home dir from the path!
cov1_barcodes_df =  pd.read_csv(gse152418_folder + 'geo/GSE155673_cov01_barcodes.tsv', sep='\t')
cov1_barcodes_df.head()

In [ ]:

# TODO @Elias Schreiner: please rename after you removed your home dir from the path!
sparse_matrix = mmread(gse152418_folder+ 'geo/GSE155673_cov01_matrix.mtx')
dense_matrix = sparse_matrix.todense()
cov1_matrix_as_df = pd.DataFrame(dense_matrix)
print(cov1_matrix_as_df.shape)
cov1_matrix_as_df.head()